In [1]:
from petsc4py import PETSc
import numpy as np

The following functions test part of the code, the results are visual only

In [2]:
def build_Q(N):
    Q = PETSc.Mat().create()
    Q.setSizes((N, N))  
    Q.setType("seqaij")  
    Q.setFromOptions()
    Q.setUp()
    return Q

def test_build_Q(test):
    if test:
        print("Test building Q")
        print("The test is OK if there is a 4x4 matrix full of zero")
        N = 4
        
        # Tested code :
        # Start
        Q = build_Q(N)
        # End
        Q.view()

In [3]:
test_build_Q(True)


Test building Q
The test is OK if there is a 4x4 matrix full of zero
Matrix has not been assembled yet


In [4]:
def create_V_storev1(size_v1, N, v1):
    Vn = PETSc.Mat().create()
    Vn.setSizes((size_v1, N))  
    Vn.setType("seqdense")  
    Vn.setFromOptions()
    Vn.setUp()    
    Vn.setValues([i for i in range(size_v1)], 0, v1, PETSc.InsertMode.INSERT_VALUES) #Vn[0] = v1
    return Vn

def test_create_V_storev1(test):
    if test:
        print("Test the creation of V and the storing of the first vector\n")
        
        N = 4
        v1 = np.array([1, -2, 3, 10, 2.2])
        print(f"The test is OK if there is a {len(v1)}x{N} matrix \nwith the first column full filled by {v1}\n")
        v1 = PETSc.Vec().createWithArray(v1)
        size_v1 = v1.getSize()

        # Tested code :
        # Start
        Vn = create_V_storev1(size_v1, N, v1)
        # End
        Vn.assemble()
        Vn.view()

In [5]:
test_create_V_storev1(True)

Test the creation of V and the storing of the first vector

The test is OK if there is a 5x4 matrix 
with the first column full filled by [ 1.  -2.   3.  10.   2.2]

Mat Object: 1 MPI process
  type: seqdense
1.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 
-2.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 
3.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 
1.0000000000000000e+01 0.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 
2.2000000000000002e+00 0.0000000000000000e+00 0.0000000000000000e+00 0.0000000000000000e+00 


In [7]:
# Create a non singular matrix to test solving part of the program
z = np.array([[1,1,0,0],[1,1,1,0],[0,1,1,1],[0,0,1,1]])
N = len(z)
Q = build_Q(N)
for i in range(N):
    v = z[i]
    Q.setValues([i for i in range(len(v))], i, v, PETSc.InsertMode.INSERT_VALUES) 
#Q.setType("seqaij")
Q.assemble()
Q.view()
rhs = np.array([1, -2, 3, 2.2])
rhs = PETSc.Vec().createWithArray(rhs)

ksp = PETSc.KSP().create()
ksp.setOperators(Q)
ksp.setType("gmres")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("superlu")

sol = rhs.copy()
sol.view()
ksp.solve(rhs, sol)
sol.view()

Mat Object: 1 MPI process
  type: seqaij
row 0: (0, 1.)  (1, 1.)  (2, 0.)  (3, 0.) 
row 1: (0, 1.)  (1, 1.)  (2, 1.)  (3, 0.) 
row 2: (0, 0.)  (1, 1.)  (2, 1.)  (3, 1.) 
row 3: (0, 0.)  (1, 0.)  (2, 1.)  (3, 1.) 
Vec Object: 1 MPI process
  type: seq
1.
-2.
3.
2.2
Vec Object: 1 MPI process
  type: seq
0.2
0.8
-3.
5.2
